In [1]:
import pickle

In [2]:
with open('../Datasets/astral-scopedom-seqres-gd-sel-gs-bib-40-2.07.fa','r') as f:
    lines = f.readlines()
    len_lines = len(lines)

In [ ]:
fold_protein_dict = {}
protein_dict = {} 

for i in range(len_lines):
    line = lines[i]
    if line[0] == '>':
        num += 1
        if i != 0:
                
            if not fold in fold_dict.keys():
                fold_dict[fold] = 1
            else:
                fold_dict[fold] += 1
                
            length = len(seq)
            
            if not length in length_dict.keys():
                length_dict[length] = 1
            else:
                length_dict[length] += 1
                
        fold = line.split(' ')[1]
        fold = '.'.join(fold.split('.')[0:2])
        clas = fold[0]
        seq = ''
    else:
        seq += line.strip('\n')

if not clas in class_dict.keys():
    class_dict[clas] = 1
else:
    class_dict[clas] += 1        
        
if not fold in fold_dict.keys():
    fold_dict[fold] = 1
else:
    fold_dict[fold] += 1

### Datasets for Seq-SS models: ###

### Datasets for Seq-Fold models: ###

In [ ]:
with open('../Datasets/simple_fold_num_dict', 'rb') as handle:
    fold_dict_simple = pickle.load(handle)